<a href="https://colab.research.google.com/github/qahtanaa/OnSubGroupFairness/blob/main/GERMAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%reset -f

In [24]:
!pip install aif360

In [25]:
!pip install ucimlrepo

In [56]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
statlog_german_credit_data = fetch_ucirepo(id=144)

# data (as pandas dataframes)
X = statlog_german_credit_data.data.features
y = statlog_german_credit_data.data.targets

# metadata
print(statlog_german_credit_data.metadata)

# variable information
print(statlog_german_credit_data.variables)

{'uci_id': 144, 'name': 'Statlog (German Credit Data)', 'repository_url': 'https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data', 'data_url': 'https://archive.ics.uci.edu/static/public/144/data.csv', 'abstract': 'This dataset classifies people described by a set of attributes as good or bad credit risks. Comes in two formats (one all numeric). Also comes with a cost matrix', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1000, 'num_features': 20, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Other', 'Marital Status', 'Age', 'Occupation'], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1994, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5NC77', 'creators': ['Hans Hofmann'], 'intro_paper': None, 'additional_info': {'summary': 'Two datasets are provided.  the original dataset, in the form provided by

In [57]:
##Download the files from these links and move these files to the path mentioned below
##https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
##https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
##
##
! mv /content/german.data /usr/local/lib/python3.6/dist-packages/aif360/datasets/../data/raw/german/german.data
! mv /content/german.doc  /usr/local/lib/python3.6/dist-packages/aif360/datasets/../data/raw/german/german.doc

mv: cannot stat '/content/german.data': No such file or directory
mv: cannot stat '/content/german.doc': No such file or directory


In [58]:
import numpy as np
import pandas as pd
from aif360.metrics import utils
from scipy.sparse import issparse
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [72]:
class DataPreparation():
  """
  ........
  """
  def __init__(self, df, sensitive, label, priv, unpriv, fav, unfav, categorical):
    """
    Construct all necessary attributes for the data preparation.

    df : (pandas DataFrame) containing the data
    sensitive : (list(str)) specifying the column names of all sensitive features
    label : (str) specifying the label column
    priv : (list(dicts)) representation of the privileged groups
    unpriv : (list(dicts)) representation of the unprivileged groups
    fav : (str/int/..) value representing the favorable label
    unfav : (str/int/..) value representing the unfavorable label
    categorical : (list(str)) (optional) specifying column names of categorical features
    """
    self.df = df
    self.sensitive = sensitive
    self.label = label
    self.priv = priv
    self.unpriv = unpriv
    self.fav = fav
    self.unfav = unfav
    self.categorical = categorical


  def detect_missing_values(self):
      """
      Detect rows with missing values and remove them from the DataFrame.
      """
      initial_rows = len(self.df)
      self.df = self.df.dropna()
      removed_rows = initial_rows - len(self.df)

      if removed_rows > 0:
          print(f"Detected {removed_rows} rows with missing values. Removed them.")
      else:
          print("No missing values detected.") #pass

  def binary_label(self):
    """
    Check that the decision label is made of two values, change it as a binary representation
    where favorable label = 1, unfavorable label = 0.
    """
    number_label_values = self.df[self.label].nunique()
    if number_label_values == 2:
      print(f"The '{self.label}' column has only two unique values.")
      self.df[self.label].replace([self.unfav, self.fav], [0, 1], inplace=True)
    else:
      print(f"The '{self.label}' column does not have exactly two unique values, as it should.")

  def find_categorical_attributes(self):
    """
    find the categorical attributes
    """
    attribute_types = {}

    for column in self.df.columns:
        # Skip the 'Group' column
        if column == 'Group':
            continue
        # if the column has already been classified as categorical by the user, leave it cat
        elif column in self.categorical:
            attribute_types[column] = 'Categorical'
            continue
        # if the column has only two distinguished values, consider it categorical
        elif self.df[column].nunique() == 2:
          attribute_types[column] = 'Categorical'
          continue
        else: #if it's not the group column, or if it's not been classified as categorical by the user, check every value of the column
          num_float = 0
          num_text = 0
          thresh = 0.99
          num_att_in_column = len(self.df[column])

          for value in self.df[column]:  # Accessing all values in the column
            # Attempt to convert the value to a float
            try:
              float(value)
              num_float += 1
              continue  # Move to the next value
            except ValueError:
              pass  # If it's not a float, continue to the next check
            # If it's not an integer or a float, consider it as text
            num_text += 1

          # now see if it's categorical or numerical
          if num_float / num_att_in_column > thresh:
            attribute_types[column] = 'Numerical'
            continue
          else:
            attribute_types[column] = 'Categorical'
    return attribute_types


  def create_group_column(self):
    """
    Create a 'Group' column in the DataFrame based on protected attributes, privileged/unprivileged conditions, and label. Then drop
    the protected attributes and label columns because those informations are already present in the Group column
    """
    group_combinations = pd.MultiIndex.from_product([self.df[sensitive].unique() for sensitive in self.sensitive] + [self.df[self.label].unique()], names=self.sensitive + [self.label])
    #print(group_combinations)
    print(list(enumerate(group_combinations)))
    # Create a mapping between group combinations and their corresponding numbers
    group_mapping = {group: idx for idx, group in enumerate(group_combinations)}
    reverse_group_mapping = {idx: group for group, idx in group_mapping.items()}  # Create reverse mapping
    # Apply the mapping to create a new column in the DataFrame
    self.df['Group'] = pd.MultiIndex.from_frame(self.df[self.sensitive + [self.label]]).map(group_mapping)

    return reverse_group_mapping #enumerate(group_combinations)

  def prepare(self):
    """
    Perform all preprocessing steps.
    """
    self.detect_missing_values()
    self.binary_label()
    self.create_group_column()
    self.find_categorical_attributes()
    #self.normalization()
    #self.make_numerical()
    return self

In [60]:
# Define a function to convert age values to categories
def convert_age_to_category(age):
    if age >= 25:
        return 1 #'Adult'
    else:
        return 2 #'Youth'

def convert_sex(sex):
  if sex == 'A92':
    return 'female'
  else:
    return 'male'

# Apply the function to the 'Attribute13' column
X_new = X.copy()
X_new['Attribute9'] = X['Attribute9'].apply(convert_sex) #Attribute9 == sex
X_new['Attribute13'] = X['Attribute13'].apply(convert_age_to_category) #Attribute13 == age

df = pd.concat([X_new, y], axis=1)
#print(df.iloc[0:50, 8])


#count the number of columns in the dataframe
num_of_columns = len(df.columns)
print("Number of columns:", num_of_columns)


Number of columns: 21


In [74]:
#use the DataPreparation class to preprocess the dataframe
data_prep = DataPreparation(df, ['Attribute9','Attribute13'], 'class', ['male', 1], ['female', '2'], 1, 2, ['Attribute1'])
data_prep.prepare()

# use this reset index to have indices from 0 to len(df)
data_prep.df = data_prep.df.reset_index(drop=True)
print(data_prep.df)


No missing values detected.
The 'class' column has only two unique values.
[(0, ('male', 1, 1)), (1, ('male', 1, 0)), (2, ('male', 2, 1)), (3, ('male', 2, 0)), (4, ('female', 1, 1)), (5, ('female', 1, 0)), (6, ('female', 2, 1)), (7, ('female', 2, 0))]
    Attribute1  Attribute2 Attribute3 Attribute4  Attribute5 Attribute6  \
0          A11           6        A34        A43        1169        A65   
1          A12          48        A32        A43        5951        A61   
2          A14          12        A34        A46        2096        A61   
3          A11          42        A32        A42        7882        A61   
4          A11          24        A33        A40        4870        A61   
..         ...         ...        ...        ...         ...        ...   
995        A14          12        A32        A42        1736        A61   
996        A11          30        A32        A41        3857        A61   
997        A14          12        A32        A43         804        A61  

In [83]:
# Call the method to find categorical indices
attribute_types = data_prep.find_categorical_attributes()
attribute_types['Attribute18'] = 'Numerical'

print("dictionary 'Attribute' : 'Numerical/Categorical' \n ", attribute_types)

# Initialize cat_features list
cat_features = []

# Iterate over attribute names and determine whether they are categorical or numerical
for attr in attribute_types:
    cat_features.append(attribute_types[attr] == 'Categorical')

del attr

print('boolean categorical features vector:\n', cat_features)


dictionary 'Attribute' : 'Numerical/Categorical' 
  {'Attribute1': 'Categorical', 'Attribute2': 'Numerical', 'Attribute3': 'Categorical', 'Attribute4': 'Categorical', 'Attribute5': 'Numerical', 'Attribute6': 'Categorical', 'Attribute7': 'Categorical', 'Attribute8': 'Numerical', 'Attribute9': 'Categorical', 'Attribute10': 'Categorical', 'Attribute11': 'Numerical', 'Attribute12': 'Categorical', 'Attribute13': 'Categorical', 'Attribute14': 'Categorical', 'Attribute15': 'Categorical', 'Attribute16': 'Numerical', 'Attribute17': 'Categorical', 'Attribute18': 'Numerical', 'Attribute19': 'Categorical', 'Attribute20': 'Categorical', 'class': 'Categorical'}
boolean categorical features vector:
 [True, False, True, True, False, True, True, False, True, True, False, True, True, True, True, False, True, False, True, True, True]


In [84]:
reverse_group_mapping = data_prep.create_group_column()

[(0, ('male', 1, 1)), (1, ('male', 1, 0)), (2, ('male', 2, 1)), (3, ('male', 2, 0)), (4, ('female', 1, 1)), (5, ('female', 1, 0)), (6, ('female', 2, 1)), (7, ('female', 2, 0))]


In [85]:
theoretical_num_groups = len(reverse_group_mapping)

DISTANCES

GOWER, HEOM, HVDM


In [87]:
!pip install gower
!pip install distython
from distython import HEOM
from distython import HVDM
import gower

In [88]:
def gower_distance(ndary, cat_features_boolean, n=5):
  """
  ndary: ndarray, that if the dataframe X precedentemente converted to numpy, it has all columns beside the Group one (yes, it also has
        the label column)
  cat_features: array of T/F where True = the attribute is categorical, False = the attribute is numerical
  n = number or closest neighbors
  """
  X = ndary
  cat_features = cat_features_boolean
  #n+1 because in the matrix with indices, at the first position there's the point itself
  n = n+1

  #compute the matrix of the distances using Gower
  matrix_dist_gower = gower.gower_matrix(X, cat_features = cat_features)

  #compute the matrix with top-n neighbors and the distance values
  num_rows_gower = X.shape[0]

  # Initialize a list to store the results
  matrix_neighbors_gower = []

  # Loop through each row
  for i in range(num_rows_gower):
    top_n_neighbors_gower = gower.gower_topn(X[i:i+1, :], X[:, :], cat_features = cat_features, n=6)
    matrix_neighbors_gower.append(top_n_neighbors_gower)

  # Convert the list of dictionaries to a matrix
  matrix_neighbors_gower_index = np.array([item['index'] for item in matrix_neighbors_gower])
  matrix_neighbors_gower_values = np.array([item['values'] for item in matrix_neighbors_gower])

  return matrix_neighbors_gower_index, matrix_neighbors_gower_values


def heom_distance(ndary, cat_features_boolean, n=5):
  """
  later
  """
  X = ndary
  n = n+1

  # convert the values of the cat_features from boolean to binary: 1 if categorical, 0 if numerical
  cat_attr_binary = [1 if x else 0 for x in cat_features_boolean]
  nan_eqv = 12345

  #compute the matrix of the distances using HEOM
  matrix_dist_HEOM = HEOM(X, cat_attr_binary, nan_equivalents = [nan_eqv])

  matrix_distances = np.zeros((len(X), len(X)))
  distances = []
  k = 0

  #compute the distances using HEOM metric
  for i in X:
      for j in X:
          i = np.asarray(i)
          i = i.astype(np.float64)
          j = np.asarray(j)
          j = j.astype(np.float64)
          dist = matrix_dist_HEOM.heom(i, j)
          distances.append(dist)

      matrix_distances[k, :] = distances
      k += 1
      distances = []

  #compute the matrices with n-NN indices and values
  matrix_ind_val = []
  for i in range(len(X)):
    dm = matrix_distances[i,:]
    top_n = smallest_indices(np.nan_to_num(dm, nan=1),6)
    matrix_ind_val.append(top_n)

  matrix_neighbors_heom_index = np.array([item['index'] for item in matrix_ind_val])
  matrix_neighbors_heom_values = np.array([item['values'] for item in matrix_ind_val])

  return matrix_neighbors_heom_index, matrix_neighbors_heom_values


def hvdm_distance(ndary, y_ix, cat_features_boolean, n=5):
  """
  y_ix = [n] is a 1-element array what represent the index of the target variable/laber
  """
  X = ndary
  n = n+1

  # convert the values of the cat_features from boolean to binary: 1 if categorical, 0 if numerical
  cat_attr_binary = [1 if x else 0 for x in cat_features_boolean]
  nan_eqv = 12345

  #compute the matrix of the distances using HEOM
  matrix_dist_HVDM = HVDM(X, y_ix, cat_attr_binary, nan_equivalents = [nan_eqv], normalised="std")

  matrix_distances = np.zeros((len(X), len(X)))
  distances = []
  k = 0

  #compute the distances using HEOM metric
  for i in X:
      for j in X:
          i = np.asarray(i)
          i = i.astype(np.float64)
          j = np.asarray(j)
          j = j.astype(np.float64)
          dist = matrix_dist_HVDM.hvdm(i, j)
          distances.append(dist)

      matrix_distances[k, :] = distances
      k += 1
      distances = []

  #compute the matrices with n-NN indices and values
  matrix_ind_val = []
  for i in range(len(X)):
    dm = matrix_distances[i,:]
    top_n = smallest_indices(np.nan_to_num(dm, nan=1),6)
    matrix_ind_val.append(top_n)

  matrix_neighbors_hvdm_index = np.array([item['index'] for item in matrix_ind_val])
  matrix_neighbors_hvdm_values = np.array([item['values'] for item in matrix_ind_val])

  return matrix_neighbors_hvdm_index, matrix_neighbors_hvdm_values



def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    #n += 1
    flat = np.nan_to_num(ary.flatten(), nan=999)
    indices = np.argpartition(-flat, -n)[-n:]
    indices = indices[np.argsort(flat[indices])]
    #indices = np.delete(indices,0,0)
    values = flat[indices]
    return {'index': indices, 'values': values}

In [152]:
# FIRST OF ALL, Encode the dataframe so that the categorical attributes are converted to numbers, but still treated like categorical.
X = data_prep.df.drop(columns=['Group'], axis=1).copy()
#print(X)
encoder_dict = dict()
columns_categorical = X.columns[cat_features]

# for each categorical column, encode it
for column in columns_categorical:
  le = LabelEncoder()
  X[column] = le.fit_transform(X[column].values)
  mapping = dict(zip(le.classes_, range(len(le.classes_))))
  encoder_dict[column] = mapping
#print(X)

# Invert the boolean values in the list using list comprehension
numerical_features = [not feature for feature in cat_features]
# Use the inverted list to select numerical columns
columns_numerical = X.columns[numerical_features]

# int columns to float, otherwise gower has problems
for column in columns_numerical:
  X[column] = X[column].astype(float)

# convert it to numpy so that distython doesn't cry
X = X.to_numpy()

# compute the matrices you need
closest_index_gower, closest_values_gower = gower_distance(X, cat_features,5)
closest_index_heom, closest_values_heom = heom_distance(X, cat_features,5)
#################### y_ix to be found automatically
n = 6 #index of label attribute in the matrix
y_ix = [n]
closest_index_hvdm, closest_values_hvdm = hvdm_distance(X, y_ix, cat_features,5)

In [ ]:
# to select one of the 3 distances
y_ix = [6]
distance = 'HEOM' #'HEOM', 'GOWER'
print(distance)
if distance == 'HVDM':
  closest_index_hvdm, closest_values_hvdm = hvdm_distance(X, y_ix, cat_features,5)
  print(closest_index_hvdm, '\n',closest_values_hvdm)
if distance == 'HEOM':
  closest_index_heom, closest_values_heom = heom_distance(X, cat_features,5)
  print(closest_index_heom, '\n',closest_values_heom)
if distance == 'GOWER':
  closest_index_gower, closest_values_gower = gower_distance(X, cat_features,5)
  print(closest_index_gower, '\n',closest_values_gower)

In [155]:
# matrices index close to each other
print('HVDM \t\t\t HEOM \t\t\t GOWER')
for i in range(len(closest_index_hvdm)):
    print(f"{closest_index_hvdm[i]}\t{closest_index_heom[i]}\t{closest_index_gower[i]}")


HVDM 			 HEOM 			 GOWER
[  0 781 851 183 894 628]	[  0 250 177 518 744 380]	[  0  61 135 807 177 115]
[  1 521 308 724 614 722]	[  1 706 308 332 521  11]	[  1 521 308 677  39 576]
[  2 709 527 422 275 241]	[  2 527 204 422 275 241]	[  2 527 422 234 943 204]
[  3 239 316 460 874 916]	[  3 239 316 460 874 501]	[  3 460 239 316 501 286]
[  4 814 853 522 978 265]	[  4 814 853 522 545 978]	[  4 522 814 545 624 853]
[  5 829 685 886 881 791]	[  5 685 895 881 791 829]	[  5 685 881 681  87 511]
[  6 786 164  46 502 493]	[  6 253  16 100 497 567]	[  6 164 786 283  46 452]
[  7 390 224 880  32  38]	[  7 893  32 291  51 937]	[  7 511 291 425 937 224]
[  8 629 816 502   6 478]	[  8 452 629 867 816  90]	[  8 430  84 995 629 478]
[  9 320 578 474  35  76]	[  9 320 935 980 578 474]	[  9 320 578 128 474  13]
[ 10 500 601 166 747 647]	[ 10 707 601 462 747 651]	[ 10 952 500 809 747 398]
[ 11 922 471 842 155 249]	[ 11 922 471 155 610 359]	[ 11 922 504 471 359 926]
[ 12 817 565 258 196  47]	[ 12 480 565 1

In [156]:
complete_df = data_prep.df
print(complete_df)

    Attribute1  Attribute2 Attribute3 Attribute4  Attribute5 Attribute6  \
0          A11           6        A34        A43        1169        A65   
1          A12          48        A32        A43        5951        A61   
2          A14          12        A34        A46        2096        A61   
3          A11          42        A32        A42        7882        A61   
4          A11          24        A33        A40        4870        A61   
..         ...         ...        ...        ...         ...        ...   
995        A14          12        A32        A42        1736        A61   
996        A11          30        A32        A41        3857        A61   
997        A14          12        A32        A43         804        A61   
998        A11          45        A32        A43        1845        A61   
999        A12          45        A34        A41        4576        A62   

    Attribute7  Attribute8 Attribute9 Attribute10  ...  Attribute13  \
0          A75           4  

_________________Type  of points: Safe, Borderline, Rare, Outliers
"Types of minority class examples and their influence
on learning classifiers from imbalanced data"

In [157]:
#drop the first column of the matrix, of course the closest point to i is i
closest_index_heom = closest_index_heom[:,1:]


# matrice che per ogni gruppo (riga) ti da il numero di [S,B,R,O]
num_groups = theoretical_num_groups
matrcie_heom = np.zeros((num_groups,6))

for sample in range(len(closest_index_heom)):
  sample_group = complete_df.iloc[sample]['Group']
  same_group_neighbors =0

  for column_value in closest_index_heom[sample,:]:
    #if the group of the examinated point is the same of the group of its 5 closest neighbors
    if sample_group == complete_df.iloc[column_value]['Group']:
      #count the number of neighbors from the same group of the examinated sample
      same_group_neighbors = same_group_neighbors + 1
  matrcie_heom[sample_group, same_group_neighbors] = matrcie_heom[sample_group, same_group_neighbors] +1
  #print('indice del punto (sample):', sample,'gruppo del sample:',sample_group, 'numero vicini dello stesso gruppo:', same_group_neighbors)
print(matrcie_heom)

[[  0.   0.   2.   6.  61. 390.]
 [  1.   3.   5.  27.  51.  79.]
 [  3.   7.  10.   9.   8.   3.]
 [  1.   4.   7.   8.   4.   1.]
 [  0.   3.  11.  24.  51.  64.]
 [  1.   8.   9.  16.  25.  14.]
 [  0.   2.   9.  15.  11.  11.]
 [  0.   2.   3.   8.  12.  11.]]


In [158]:
#drop the first column of the matrix, of course the closest point to i is i
closest_index_hvdm = closest_index_hvdm[:,1:]

# matrice che per ogni gruppo (riga) ti da il numero di [S,B,R,O]
num_groups = theoretical_num_groups
matrcie_hvdm = np.zeros((num_groups,6))

for sample in range(len(closest_index_hvdm)):
  #sto considerando una riga in particolare, con indice riga pari a sample, ed estraggo il gruppo
  sample_group = complete_df.iloc[sample]['Group']
  same_group_neighbors =0

  for column_value in closest_index_hvdm[sample,:]:
    #if the group of the examinated point is the same of the group of its 5 closest neighbors
    if sample_group == complete_df.iloc[column_value]['Group']:
      #count the number of neighbors from the same group of the examinated sample
      same_group_neighbors = same_group_neighbors + 1
  matrcie_hvdm[sample_group, same_group_neighbors] = matrcie_hvdm[sample_group, same_group_neighbors] +1
  #print('indice del punto (sample):', sample,'gruppo del sample:',sample_group, 'numero vicini dello stesso gruppo:', same_group_neighbors)
print(matrcie_hvdm)

[[  0.   0.   0.   1.   5. 453.]
 [  0.   0.   2.   8.  30. 126.]
 [  0.   2.   6.   6.   5.  21.]
 [  0.   3.   5.   1.   5.  11.]
 [  1.   0.   2.   1.  21. 128.]
 [  0.   3.   1.   9.  13.  47.]
 [  0.   0.   1.   6.  11.  30.]
 [  0.   0.   2.   1.   3.  30.]]


In [159]:
#drop the first column of the matrix, of course the closest point to i is i
closest_index_gower = closest_index_gower[:,1:]

# matrice che per ogni gruppo (riga) ti da il numero di [S,B,R,O]
num_groups = theoretical_num_groups
matrcie_gower = np.zeros((num_groups,6))

for sample in range(len(closest_index_gower)):
  #sto considerando una riga in particolare, con indice riga pari a sample, ed estraggo il gruppo
  sample_group = complete_df.iloc[sample]['Group']
  same_group_neighbors =0

  for column_value in closest_index_gower[sample,:]:
    #if the group of the examinated point is the same of the group of its 5 closest neighbors
    if sample_group == complete_df.iloc[column_value]['Group']:
      #count the number of neighbors from the same group of the examinated sample
      same_group_neighbors = same_group_neighbors + 1
  matrcie_gower[sample_group, same_group_neighbors] = matrcie_gower[sample_group, same_group_neighbors] +1
  #print('indice del punto (sample):', sample,'gruppo del sample:',sample_group, 'numero vicini dello stesso gruppo:', same_group_neighbors)
print(matrcie_gower)

[[  0.   0.   4.  27. 117. 311.]
 [  2.  12.  26.  53.  54.  19.]
 [  6.   3.  15.  11.   4.   1.]
 [  4.   6.   7.   5.   3.   0.]
 [  4.   8.  37.  42.  41.  21.]
 [  6.   8.  21.  15.  16.   7.]
 [  0.  11.  12.  10.  10.   5.]
 [  1.   3.   9.   7.  12.   4.]]


In [164]:
# MATRIX [#GROUPS, #TAXONOMY], where taxonomy is S,B,R,O
# Define constants for the taxonomy categories
#HEOM
SAFE = 0
BORDERLINE = 1
RARE = 2
OUTLIER = 3

# Initialize matrix_taxonomy with dtype=int for clarity
matrix_taxonomy = np.zeros((num_groups, 4), dtype=int)

# Loop through each sample
for sample, neighbors_indices in enumerate(closest_index_heom):
    sample_group = complete_df.iloc[sample]['Group']
    same_group_neighbors = 0
    same_group_neighbor_indices = []

    # Count neighbors from the same group and collect their indices
    for neighbor_index in neighbors_indices:
        if sample_group == complete_df.iloc[neighbor_index]['Group']:
            same_group_neighbors += 1
            same_group_neighbor_indices.append(neighbor_index)

    # Classify based on neighbor counts
    if same_group_neighbors == 5 or same_group_neighbors == 4:
        classification = SAFE
    elif same_group_neighbors == 3 or same_group_neighbors == 2:
        classification = BORDERLINE
    elif same_group_neighbors == 1:
        # Check if the neighbor has either 0:5 or 1:4 neighbors from the same class
        solo_neighbor = same_group_neighbor_indices[0]
        neighbors_of_solo_neighbor = closest_index_heom[solo_neighbor]

        same_class_neighbors = sum(
            1 for neighbor_index in neighbors_of_solo_neighbor if complete_df.iloc[neighbor_index]['Group'] == sample_group
        )

        if same_class_neighbors == 0 or same_class_neighbors == 1:
            classification = RARE
        else:
            classification = BORDERLINE
    elif same_group_neighbors == 0:
        classification = OUTLIER
    else:
        classification = BORDERLINE  # If not falling into any specific category, consider as Borderline

    # Update the taxonomy matrix
    matrix_taxonomy[sample_group, classification] += 1
    # print('Index of the sample:', sample, 'Group of the sample:', sample_group, 'Number of neighbors from the same group:', same_group_neighbors)

#print(matrix_taxonomy)

# Define the taxonomy labels
taxonomy_labels = ['S', 'B', 'R', 'O']

# Create a DataFrame from matrix_taxonomy
taxonomy_df_heom = pd.DataFrame(matrix_taxonomy, columns=taxonomy_labels)

# Add a column for group numbers
taxonomy_df_heom['Group'] = range(num_groups)

# Set 'Group' column as the index
taxonomy_df_heom.set_index('Group', inplace=True)

# Print the DataFrame
taxonomy_df_heom


,S,B,R,O
Group,,,,
0,451,8,0,0
1,130,34,1,1
2,11,22,4,3
3,5,19,0,1
4,115,38,0,0
5,39,33,0,1
6,22,26,0,0
7,23,12,1,0


In [165]:
# MATRIX [#GROUPS, #TAXONOMY], where taxonomy is S,B,R,O
# Define constants for the taxonomy categories
#GOWER
SAFE = 0
BORDERLINE = 1
RARE = 2
OUTLIER = 3

# Initialize matrix_taxonomy with dtype=int for clarity
matrix_taxonomy = np.zeros((num_groups, 4), dtype=int)

# Loop through each sample
for sample, neighbors_indices in enumerate(closest_index_gower):
    sample_group = complete_df.iloc[sample]['Group']
    same_group_neighbors = 0
    same_group_neighbor_indices = []

    # Count neighbors from the same group and collect their indices
    for neighbor_index in neighbors_indices:
        if sample_group == complete_df.iloc[neighbor_index]['Group']:
            same_group_neighbors += 1
            same_group_neighbor_indices.append(neighbor_index)

    # Classify based on neighbor counts
    if same_group_neighbors == 5 or same_group_neighbors == 4:
        classification = SAFE
    elif same_group_neighbors == 3 or same_group_neighbors == 2:
        classification = BORDERLINE
    elif same_group_neighbors == 1:
        # Check if the neighbor has either 0:5 or 1:4 neighbors from the same class
        solo_neighbor = same_group_neighbor_indices[0]
        neighbors_of_solo_neighbor = closest_index_gower[solo_neighbor]

        same_class_neighbors = sum(
            1 for neighbor_index in neighbors_of_solo_neighbor if complete_df.iloc[neighbor_index]['Group'] == sample_group
        )

        if same_class_neighbors == 0 or same_class_neighbors == 1:
            classification = RARE
        else:
            classification = BORDERLINE
    elif same_group_neighbors == 0:
        classification = OUTLIER
    else:
        classification = BORDERLINE  # If not falling into any specific category, consider as Borderline

    # Update the taxonomy matrix
    matrix_taxonomy[sample_group, classification] += 1
    # print('Index of the sample:', sample, 'Group of the sample:', sample_group, 'Number of neighbors from the same group:', same_group_neighbors)

#print(matrix_taxonomy)

# Define the taxonomy labels
taxonomy_labels = ['S', 'B', 'R', 'O']

# Create a DataFrame from matrix_taxonomy
taxonomy_df_gower = pd.DataFrame(matrix_taxonomy, columns=taxonomy_labels)

# Add a column for group numbers
taxonomy_df_gower['Group'] = range(num_groups)

# Set 'Group' column as the index
taxonomy_df_gower.set_index('Group', inplace=True)

# Print the DataFrame
taxonomy_df_gower


,S,B,R,O
Group,,,,
0,428,31,0,0
1,73,87,4,2
2,5,28,1,6
3,3,16,2,4
4,62,86,1,4
5,23,43,1,6
6,15,28,5,0
7,16,19,0,1


In [166]:
# MATRIX [#GROUPS, #TAXONOMY], where taxonomy is S,B,R,O
# Define constants for the taxonomy categories
#HVDM
SAFE = 0
BORDERLINE = 1
RARE = 2
OUTLIER = 3

# Initialize matrix_taxonomy with dtype=int for clarity
matrix_taxonomy = np.zeros((num_groups, 4), dtype=int)

# Loop through each sample
for sample, neighbors_indices in enumerate(closest_index_hvdm):
    sample_group = complete_df.iloc[sample]['Group']
    same_group_neighbors = 0
    same_group_neighbor_indices = []

    # Count neighbors from the same group and collect their indices
    for neighbor_index in neighbors_indices:
        if sample_group == complete_df.iloc[neighbor_index]['Group']:
            same_group_neighbors += 1
            same_group_neighbor_indices.append(neighbor_index)

    # Classify based on neighbor counts
    if same_group_neighbors == 5 or same_group_neighbors == 4:
        classification = SAFE
    elif same_group_neighbors == 3 or same_group_neighbors == 2:
        classification = BORDERLINE
    elif same_group_neighbors == 1:
        # Check if the neighbor has either 0:5 or 1:4 neighbors from the same class
        solo_neighbor = same_group_neighbor_indices[0]
        neighbors_of_solo_neighbor = closest_index_hvdm[solo_neighbor]

        same_class_neighbors = sum(
            1 for neighbor_index in neighbors_of_solo_neighbor if complete_df.iloc[neighbor_index]['Group'] == sample_group
        )

        if same_class_neighbors == 0 or same_class_neighbors == 1:
            classification = RARE
        else:
            classification = BORDERLINE
    elif same_group_neighbors == 0:
        classification = OUTLIER
    else:
        classification = BORDERLINE  # If not falling into any specific category, consider as Borderline

    # Update the taxonomy matrix
    matrix_taxonomy[sample_group, classification] += 1
    # print('Index of the sample:', sample, 'Group of the sample:', sample_group, 'Number of neighbors from the same group:', same_group_neighbors)

#print(matrix_taxonomy)

# Define the taxonomy labels
taxonomy_labels = ['S', 'B', 'R', 'O']

# Create a DataFrame from matrix_taxonomy
taxonomy_df_hvdm = pd.DataFrame(matrix_taxonomy, columns=taxonomy_labels)

# Add a column for group numbers
taxonomy_df_hvdm['Group'] = range(num_groups)

# Set 'Group' column as the index
taxonomy_df_hvdm.set_index('Group', inplace=True)

# Print the DataFrame
taxonomy_df_hvdm


,S,B,R,O
Group,,,,
0,458,1,0,0
1,156,10,0,0
2,26,14,0,0
3,16,7,2,0
4,149,3,0,1
5,60,13,0,0
6,41,7,0,0
7,33,3,0,0


In [181]:

# Concatenate the DataFrames horizontally
concatenated_df = pd.concat([taxonomy_df_hvdm, taxonomy_df_heom, taxonomy_df_gower], axis=1)
#print('\t HVDM, \t HEOM, \t GOWER')
# Print the concatenated DataFrame
concatenated_df


,S,B,R,O,S,B,R,O,S,B,R,O
Group,,,,,,,,,,,,
0,458,1,0,0,451,8,0,0,428,31,0,0
1,156,10,0,0,130,34,1,1,73,87,4,2
2,26,14,0,0,11,22,4,3,5,28,1,6
3,16,7,2,0,5,19,0,1,3,16,2,4
4,149,3,0,1,115,38,0,0,62,86,1,4
5,60,13,0,0,39,33,0,1,23,43,1,6
6,41,7,0,0,22,26,0,0,15,28,5,0
7,33,3,0,0,23,12,1,0,16,19,0,1
